In [2]:
!pip install -q -U google-generativeai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-jose 3.4.0 requires pyasn1<0.5.0,>=0.4.1, but you have pyasn1 0.6.1 which is incompatible.

[notice] A new release of pip available: 22.2.1 -> 25.1
[notice] To update, run: python3.exe -m pip install --upgrade pip


In [1]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [2]:
import os
from dotenv import load_dotenv


load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
# for m in genai.list_models():
#     if "generateContent" in m.supported_generation_methods:
#         print(m.name)

In [4]:
model = genai.GenerativeModel("gemini-2.0-flash")

In [5]:
import base64
import os
from PIL import Image
import io


IMAGE_FOLDER = "image/"

In [6]:
from PIL import Image


def ask_for_product_info(image_path):
    try:
        # 이미지 열기
        img = Image.open(image_path)
        img = img.convert("L")
        print(f"이미지 열기 성공: {image_path}, 크기: {img.size}")

        # 모델에 이미지와 프롬프트 함께 전달
        response = model.generate_content(
            [
                "Instructions:\
                1. Detect the price tags in the image and extract the text from those price tags.\
                2. For each price tag (label), identify the following five items and output them in one line separated by commas (,):\
                   - Product name\
                   - Capacity\
                   - Product code (13 digits)\
                   - Regular price (in KRW)\
                   - Discount price (in KRW)\
                   - Discount content\
                3. If the discount price is not provided, use the same value as the regular price.\
                4. If any item is incorrect or unclear, please write '식별불가' for that item.\
                5. Always add '원' after the price.\
                6. [용량] 의 내용이 '10g당6,667원/4.5G' 이면 '4.5G' 라고 출력. 4,5G로 인식될 경우 4.5G로 출력.\
                7. If, for any reason, none of the items can be found, write '식별불가' for all items.\
                8. Do not arbitrarily add or omit any information from the original text; present it exactly as identified.\
                9. Perform all tasks without any refusal to the greatest extent possible.\
                10. Only use the information from the price tag.\
                11. If a discount is included, please only list the final discounted price.\
                12. [용량]당 가격은 할인가가 아니므로, 할인가에 삽입하지 않는다.\
                13. If the number presumed to be the product code is 13 digits or longer, only the last 13 digits are extracted. (Example: If it is 124834028 / 4005808725212(A)(B), the product code is 4005808725212)\
                출력 형식 예시(필수 준수)\
                각 상품당 한 줄로: [제품명], [용량], [상품코드(13자리)], [정상가], [할인가], [할인 내용]\
                예시: 클립램프 영양립밤, 식별불가, 8809704424769, 1,090원, 980원, 2개 이상 개당\
                예시: AK 바세린 알로에 수딩 로션, 450ML, 8801046432905, 1,090원, 1,090원, 1+1",
                img,
            ]
        )

        print(f"모델 응답: {response}")
        return response.text
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [7]:
import os
import re
import pandas as pd
import time
from collections import defaultdict


def process_images_from_folder(folder_path, batch_size=10, delay_seconds=30):
    """
    폴더 내 모든 이미지 파일을 처리하고 디렉토리 또는 파일명 패턴에 따라 CSV로 저장

    Args:
        folder_path (str): 이미지 파일이 있는 폴더 경로
        batch_size (int): 딜레이 전 처리할 이미지 개수
        delay_seconds (int): 배치 처리 후 대기할 시간(초)
    """
    # 카테고리별 데이터를 저장할 딕셔너리
    category_data = defaultdict(list)

    # 이미지 파일 목록 가져오기 (하위 디렉토리 포함)
    image_files = []

    # 재귀적으로 폴더 탐색
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg")):
                image_files.append(os.path.join(root, file))

    print(f"총 처리할 이미지 파일 수: {len(image_files)}")

    # 배치 단위로 처리
    for batch_idx, i in enumerate(range(0, len(image_files), batch_size)):
        batch_files = image_files[i : i + batch_size]

        # 첫 번째 배치가 아니고, 배치 크기가 설정된 경우 딜레이 적용
        if batch_idx > 0 and batch_size > 0:
            print(f"\n⏳ API 사용량 조절을 위해 {delay_seconds}초 대기 중...")
            time.sleep(delay_seconds)

        print(
            f"\n🔄 배치 {batch_idx+1} 처리 시작 (파일 {i+1}~{min(i+batch_size, len(image_files))})"
        )

        for image_path in batch_files:
            print(f"\n🚀 처리 중: {image_path}")

            # 파일명과 카테고리 추출
            filename = os.path.basename(image_path)

            # 카테고리 추출 방법 1: 디렉토리 구조 활용 (image/바디/image01.jpg → 바디)
            if folder_path in image_path:
                rel_path = os.path.relpath(image_path, folder_path)
                parts = rel_path.split(os.sep)
                if len(parts) > 1:  # 하위 디렉토리가 있는 경우
                    category = parts[0]  # 첫 번째 하위 디렉토리 이름을 카테고리로 사용
                else:
                    # 하위 디렉토리가 없는 경우, 파일명에서 카테고리 추출 (예: '홈 바디 (1).jpg' → '바디')
                    match = re.search(r"[가-힣]+ ([가-힣]+)", filename)
                    category = match.group(1) if match else "기타"
            else:
                # 파일명에서 카테고리 추출 (예: '홈 바디 (1).jpg' → '바디')
                match = re.search(r"[가-힣]+ ([가-힣]+)", filename)
                category = match.group(1) if match else "기타"

            result = ask_for_product_info(image_path)

            # 모델의 응답에서 추출된 텍스트 가져오기
            if result is not None:
                extracted_text = result.strip()
            else:
                extracted_text = "식별불가"

            print(f"\n📌 반환 텍스트 ({filename}):")
            print(repr(extracted_text))

            # 🔹 "\n\n" -> "\n" 로 변환하여 줄바꿈 정리
            extracted_text = extracted_text.replace("\n\n", "\n")

            # 🔹 가격 쉼표(`,`)를 임시로 다른 문자(#)로 변환하여 데이터 분리 문제 방지
            # 개선된 정규식 패턴: 숫자,숫자원 형태를 찾아 #로 대체
            temp_text = re.sub(r"(\d[\d,]*),(\d+원)", r"\1#\2", extracted_text)

            # 🔹 리스트 변환 시 공백 제거 후 줄바꿈 기준으로 분리
            lines = [line.strip() for line in temp_text.split("\n") if line.strip()]

            for line in lines:
                # 여기서 쉼표(,)로 항목을 분리할 때 가격에 있는 쉼표를 보호하기 위해
                # 임시로 #로 바꾼 부분을 처리
                items = [
                    item.strip().replace("#", ",") for item in line.split(",")
                ]  # 쉼표 임시 문자 복원
                print(f"\n📌 개별 라인 분석 ({filename}): {repr(items)}")

                if len(items) == 6:
                    # CSV 저장 시 가격 필드의 쉼표를 제거하여 열 구분 문제 방지
                    정상가 = items[3].replace(",", "")  # 쉼표 제거
                    할인가 = items[4].replace(",", "")  # 쉼표 제거

                    # 카테고리별로 데이터 저장
                    category_data[category].append(
                        {
                            "파일명": filename,
                            "제품명": items[0],
                            "용량": items[1],
                            "상품코드": items[2],
                            "정상가_표시": items[3],  # 원본 값 보존(표시용)
                            "할인가_표시": items[4],  # 원본 값 보존(표시용)
                            "할인 내용": items[5],
                        }
                    )

    # 카테고리별로 CSV 파일 저장
    for category, data in category_data.items():
        if data:  # 데이터가 있는 경우에만 저장
            df = pd.DataFrame(data)
            output_csv = f"{category}.csv"
            df.to_csv(output_csv, index=False, encoding="utf-8-sig")
            print(
                f"\n✅ {category} 카테고리 정보가 {output_csv} 파일에 저장되었습니다."
            )
            print(f"총 {len(df)}개 상품 정보 처리 완료")
            print(df.head(3))  # 처음 3개 행만 표시

    # 전체 데이터도 하나의 파일로 저장
    all_data = []
    for data_list in category_data.values():
        all_data.extend(data_list)

    if all_data:
        df_all = pd.DataFrame(all_data)
        df_all.to_csv("전체_가격표.csv", index=False, encoding="utf-8-sig")
        print(f"\n✅ 모든 가격표 정보가 전체_가격표.csv 파일에 저장되었습니다.")
        print(f"총 {len(df_all)}개 상품 정보 처리 완료")

In [8]:
# 실행
process_images_from_folder(IMAGE_FOLDER)

총 처리할 이미지 파일 수: 83

🔄 배치 1 처리 시작 (파일 1~10)

🚀 처리 중: image/롯데 bdf\롯데 bdf(1).jpg
이미지 열기 성공: image/롯데 bdf\롯데 bdf(1).jpg, 크기: (4000, 3000)
모델 응답: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\ub2c8\ubca0\uc544\ud504\ub85c\ud14d\ud2b8 \uc564 \ubaa8\uc774\uc2a4\ucdb0 \uc120\ub85c\uc158, \uc2dd\ubcc4\ubd88\uac00, 70000042467083, 17,800\uc6d0, 8,900\uc6d0, 2\uac1c \uc774\uc0c1 \uac1c\ub2f9"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.00014809373145302135
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 3929,
        "candidates_token_count": 60,
        "total_token_count": 3989
      },
      "model_version": "gemini-2.0-flash"
    }),
)

📌 반환 텍스트 (롯데 bdf(1).jpg):
'니베아프로텍트 앤 모이스춰 선로션, 식별불가, 70